# Black Crow Take home

## Objective Function: 
Given a user, what is the probability that they will make a purchase in BCAI in the next 7 days.

### Understand Problem

I think I can reword the problem as: Given a Visitor_ID and today's date (day t), what is the probability they will make a purchase within 7 days (day t+7).

I can also split the problem into two cases: Either user has or hasn't made a purchase prior to day t. 

Factors that can be used to determine purchase probability:
    
1. Visitor_ID did *not* make a purchase prior to day t.
    1. User circumstances match that of Visitor_ID that have made a purchase before.
        1. Eg. Day of week, time, price of product, any active promotion, etc
    2. User characteristics match that of the Visitor_ID that has made a purchase. 
        1. Eg. Location, device, age, gender and any other personal identifiers 
    
2. Visitor_ID did make a purchase prior to day t.
    1. Previous purchase characteristics - May suggest purchase cadence
        1. Eg. Quantity bought, prices, time between purchases, total number of separate purchases
    2. Site activity on Day t - Can suggest window shopping
        1. Absent, Visit site, time spent, pages visited 
        
Note. I have yet to look at the data, so some of these features may not exist, or I may find new, more useful features. 
    

For simplicity sake, I am going to use Jan 10th, 2022 as day t.

## Assessment:

### Read both the datasets

In [394]:
import pyarrow.parquet as pq
import pandas as pd
from datetime import datetime
import json
from pandas import json_normalize

In [258]:
conversions = pq.read_table(source='conversions.parquet').to_pandas()

In [381]:
customers = pq.read_table(source='customers.parquet').to_pandas()

### Perform EDA (Exploratory Data Analysis)

#### Conversions Data

In [13]:
conversions.head

<bound method NDFrame.head of      REQUESTED_AT                                         VISITOR_ID
0      2022-01-14  db07fb0f4952db44383fb3a42e5169ef11eb23c9c5533d...
1      2022-01-12  a9b327fc32ae2c9749473b90145ba4b4a8dde11babba55...
2      2022-01-13  4d5e9ee969ae57389308cae43e2f5848871831fb9d539b...
3      2022-01-12  6e3f99c954427f9171f6d80b73ad3eab67f251adb05ad5...
4      2022-01-12  9cf66459dcdf50ce55196b41e1b6816aad6ca7abc0ade1...
...           ...                                                ...
2047   2022-01-10  0d7f7bdba545f7b4233f892d95b3a8bc58b3affa2a8b0a...
2048   2022-01-10  055770bf0bfffe9db47b4d8486c44eb941a3ff2eaf2ace...
2050   2022-01-10  b02024ec16a35bc3b105c383161afd3519d5e1a7d7c9ab...
2053   2022-01-10  80d6c7e2f82cdde5ca4e4279dd690d6ccb1ddf5c85bdd1...
2055   2022-01-10  9635a39f2e7886ec5fe9a2ccc44655df94509feb482b2e...

[1735 rows x 2 columns]>

In [19]:
# Number of unique visitor_ids
len(conversions.VISITOR_ID.drop_duplicates())
# That table only shows the first purchase date of user, 
# not every time they made a purchase

1735

In [20]:
# Whats the data dist of purchases?
conversions.groupby("REQUESTED_AT")["VISITOR_ID"].agg("count")
# Intersting, maybe there is a relationship with purchase and day of week

REQUESTED_AT
2022-01-09    113
2022-01-10    714
2022-01-11    186
2022-01-12     93
2022-01-13     69
2022-01-14     75
2022-01-15    169
2022-01-16    160
2022-01-17    156
Name: VISITOR_ID, dtype: int64

#### Customers Data: snapshot of user information for their last page view

In [28]:
len(customers)

24385

In [37]:
#Find out the number of unique values in each column
for i in customers.columns:
    print(i , len(customers[i].drop_duplicates()))
    

ID 24385
REQUESTED_AT 24379
SITE_ID 1
PAGE_VIEW_TYPE 10
PAGE_ID 22
SESSION_ID 22441
VISITOR_ID 24385
USER_AGENT 3169
BROWSER_FAMILY 14
BROWSER 60
OS_FAMILY 8
OS 41
DEVICE_FAMILY 4
CONVERSION_SUMMARIES 2074
FIRST_SEEN 18307
PAGE_VIEW_SUMMARIES 18866
SITE_NAME 1


#### Organize/track columns:

- One-to-One map to user
    - VISITOR_ID 24385 (KEEP)- Keep, don't see any duplicates or issues
    - ID 24385 (DROP)- Last page visit - Will drop, can use visitor id + date as identifier
    - REQUESTED_AT (KEEP/Get date - Investigated) 24379 - Maybe some users visites site at same time, hence few data points short
        - Found 6 times that had 2 users leave the site at the same time, spot check doesn't suggest any issue since the 2 users didn't seem to be related in any way. 
    - SESSION_ID 22441 (DROP - Investigated)- Should match user count...but it's off...why?
        - Due to being inconsistent with the defination of session ID and not providing directly useful information, I may drop this column for simplicty sake.
    
- Last page info
    - PAGE_VIEW_SUMMARIES 18866 (KEEP & Flatten- Remove any none values)
        - Are there common pathways?
    - PAGE_VIEW_TYPE 10 & PAGE_ID 22 (KEEP- Remove Unknown/Other)
        - I will remove Unknown+Other
        - What are these subtypes?
    
- Device info
    - USER_AGENT 3169 - (DROP MAYBE) 
        - It appears that the key info in this string is already flattend below, so can use those instead.
    - BROWSER_FAMILY 14 - (KEEP - Remove BOT & Unknown)
        - May need to reorganize since classes are unbalanced
    - BROWSER 60 - (KEEP - Remove BOT & Unknown)
        - May need to reorganize since classes are unbalanced
    - OS_FAMILY 8 - (KEEP - Remove UNKNOWN/UNKNOWN_MOBILE)
    - OS 41 - (KEEP - Need to reorganize into larger classes)
    - DEVICE_FAMILY 4 - (KEEP - Remove Unknown)
    
- Purchase history
    - CONVERSION_SUMMARIES 2074 - (KEEP + Flatten)
        - This is very useful info but need to flatten
    
- Activity history
    - FIRST_SEEN 18307 (KEEP + strip time)
        - I don't see the value of knowing the time the individual first visited the site, but the date is important. 
    
- Site name
    - SITE_NAME 1 (DROP)
    - SITE_ID 1 (DROP)

In [121]:
# look at FIRST_SEEN
customers.groupby(["FIRST_SEEN"])["VISITOR_ID"].agg("count")

# look at PAGE_VIEW_TYPE & PAGE_ID
customers.groupby(["PAGE_VIEW_TYPE", "PAGE_ID"])["VISITOR_ID"].agg("count")


# Look at REQUESTED_AT
visit_times = (customers.groupby("REQUESTED_AT")["VISITOR_ID"].agg("count")
               .reset_index().sort_values("VISITOR_ID", ascending=False))

visit_times[visit_times["VISITOR_ID"]>1]
customers[customers["REQUESTED_AT"]=='2022-01-10 18:14:56.748']

# look at SESSION_ID
(customers.groupby("SESSION_ID")['VISITOR_ID']
.nunique().reset_index().sort_values("VISITOR_ID", ascending = False)
)

,SESSION_ID,VISITOR_ID
11206,7fa09f73e69e970f86ebcbbed83f7bfb3fa10ed5f9d1c0...,133
21849,f90554292c6cb4bac43aca4e6b9554e90cca3256949d35...,104
4093,2e3b0e55f10ff56fb8c6721be57bbd52b391a0b6732a36...,59
19452,dd622fb0f7f92ff9bd75fe28ffc232d5499c6f409d2ba5...,38
19154,da172f0d5910f62059a78e69ee12a00f374dae942c2940...,38
...,...,...
7698,573390684cd30322550d11722f8ca446f44b8f892c7cee...,1
7697,57329305ec59bcfb41f9dff3d4c109b6b16f1a23908d58...,1
7695,572f65e644b9a850f85db9a56092a3a247174daf7b6678...,1
7693,57257ce46d31a3c2c9d315042c620b069325ffb27d773d...,1


#### Understand Visitor ID dist

In [475]:
import datetime
conversions_ss = conversions[conversions["REQUESTED_AT"] > datetime.date(2022, 1, 10)]

In [476]:
visitors_jan10 = set(customers.VISITOR_ID)
conversion_after10th = set(conversions_ss.VISITOR_ID)

print('#make purchase after 7 days', len(conversion_after10th & visitors_jan10))
print('%make purchase after 7 days', 1-len(visitors_jan10 - conversion_after10th)/len(visitors_jan10))

#make purchase after 7 days 881
%make purchase after 7 days 0.046346467462780816


In [477]:
# Will ignore visitor ids from the 9th since none of them were present
# on the 10th in the visitor data. Also, this would simplify identifying key features. 
conversions_9th = conversions[conversions["REQUESTED_AT"] < datetime.date(2022, 1, 10)]
conversions_9th = set(conversions_9th.VISITOR_ID)
len(conversions_9th - visitors_jan10) / len(conversions_9th)

0.008849557522123894

In [478]:
# Also, In practice, I would have used a moving window 
# approach to identify key features that are a strong predictor of future sales, given a date.
# But here I am using only a single window to identify key features.

# In this case, the window I am using is from Jan 11 to the 17th, using the visitors from 10th. 

In [479]:
# Visitors that converted within 7 days that visiting the site on Jan 10th
future_converters = list(visitors_jan10.intersection(conversion_after10th))
len(future_converters)

881

In [480]:
# Add future conversion flag into customers DF
customers.loc[customers['VISITOR_ID'].isin(future_converters), 'future_conversion_flag'] = 1
customers['future_conversion_flag'] = customers['future_conversion_flag'].fillna(0)

customers.groupby('future_conversion_flag')["VISITOR_ID"].agg("count")

future_conversion_flag
0.0    18128
1.0      881
Name: VISITOR_ID, dtype: int64

In [481]:
# Create flag for returning and new customers
customers["CONVERSION_SUMMARIES_len"]  = [len(str(i)) for i in customers["CONVERSION_SUMMARIES"] ]
returing_customers = (customers[customers["CONVERSION_SUMMARIES_len"]>4]
                        [["VISITOR_ID", "CONVERSION_SUMMARIES"]])

customers.loc[customers['VISITOR_ID'].isin(list(set(returing_customers["VISITOR_ID"])))
                  , 'returing_customer_flag'] = 1
customers['returing_customer_flag'] = customers['returing_customer_flag'].fillna(0)

customers.groupby('returing_customer_flag')["VISITOR_ID"].agg("count")

returing_customer_flag
0.0    16826
1.0     2183
Name: VISITOR_ID, dtype: int64

In [482]:
# Create flag for repeat customer making another purchase and new customer making first purchase
customers['future_purchase_type_flag'] = 'no_purchase' 
customers.loc[(customers.future_conversion_flag == 1) & (customers.returing_customer_flag == 1),
               'future_purchase_type_flag'] = 'returing_customer'
customers.loc[(customers.future_conversion_flag == 1) & (customers.returing_customer_flag == 0),
               'future_purchase_type_flag'] = 'new_customer'
customers.groupby('future_purchase_type_flag')["VISITOR_ID"].agg("count")

future_purchase_type_flag
new_customer           545
no_purchase          18128
returing_customer      336
Name: VISITOR_ID, dtype: int64

### Clean and preprocess the data.

In [389]:
# Drop columns
customers = customers.drop(columns=['ID', 'SESSION_ID', 'USER_AGENT', 'SITE_NAME', 'SITE_ID'])

In [390]:
# Remove rows with bad values
customers = customers[~customers["PAGE_VIEW_SUMMARIES"].isna()]
customers = customers[customers["PAGE_VIEW_TYPE"] != "UNKNOWN"]
customers = customers[customers["PAGE_ID"] != "other"]
customers = customers[~customers["BROWSER_FAMILY"].isin(["UNKNOWN", "BOT"])]
customers = customers[~customers["OS_FAMILY"].isin(["UNKNOWN"])]

In [395]:
# Update date values
customers["latest_visit_date"] = [i.date() for i in customers["REQUESTED_AT"]]
customers["first_visit_date"] = ([datetime.strptime(i.split("T")[0].replace('"', ''), "%Y-%m-%d").date() 
                                  for i in customers["FIRST_SEEN"]])

In [465]:
import numpy as np

purchase_df = pd.DataFrame()
for i in range(0, len(customers)):
    v_id = customers.iloc[i]["VISITOR_ID"]
        
    purchase_history = json_normalize(json.loads(customers["CONVERSION_SUMMARIES"].iloc[i]))
    
    if len(purchase_history)==1:
        
        purchase_history["next"] = ([ datetime.strptime(i.split(".")[0], '%Y-%m-%dT%H:%M:%S')  
                                      for i in purchase_history["requested_at"]])
        
        purchase_history["weekday"] = [i.weekday() for i in purchase_history["next"]]
        purchase_history["hour"] = [i.hour for i in purchase_history["next"]]
        
        median_weekday = np.median(purchase_history["weekday"])
        median_hour = np.median(purchase_history["hour"])
        
        purchase_count = len(purchase_history)
        mean_price = np.mean(purchase_history["conversion_value"])
        min_price = np.min(purchase_history["conversion_value"])
        max_price = np.max(purchase_history["conversion_value"])
        
        data = ([{'VISITOR_ID':v_id, 
         'purchase_count': purchase_count, 
         'mean_price': mean_price, 
         'min_price':min_price, 
         'max_price':max_price, 
         'median_weekday': median_weekday, 
         'median_hour': median_hour, 
         'median_repeat_purchase_duration':0
        }])

        purchase_df = pd.concat([purchase_df,pd.DataFrame(data)], ignore_index=True)
        
        
        
    elif len(purchase_history)>1:

        purchase_history["next"] = ([ datetime.strptime(i.split(".")[0], '%Y-%m-%dT%H:%M:%S')  
                                      for i in purchase_history["requested_at"]])

        purchase_history["weekday"] = [i.weekday() for i in purchase_history["next"]]
        purchase_history["hour"] = [i.hour for i in purchase_history["next"]]
        
        median_weekday = np.median(purchase_history["weekday"])
        median_hour = np.median(purchase_history["hour"])

        purchase_count = len(purchase_history)
        mean_price = np.mean(purchase_history["conversion_value"])
        min_price = np.min(purchase_history["conversion_value"])
        max_price = np.max(purchase_history["conversion_value"])

        purchase_history["previous"] = purchase_history["next"].shift(periods =1)
        purchase_history = purchase_history.iloc[1:]
        purchase_history["diff"] = purchase_history["next"] - purchase_history["previous"]
        purchase_history["diff_days"] = purchase_history["diff"].dt.days

        median_repeat_purchase_duration = np.median(purchase_history["diff_days"])


        data = ([{'VISITOR_ID':v_id, 
         'purchase_count': purchase_count, 
         'mean_price': mean_price, 
         'min_price':min_price, 
         'max_price':max_price, 
         'median_weekday': median_weekday, 
         'median_hour': median_hour, 
         'median_repeat_purchase_duration':median_repeat_purchase_duration 
        }])

        purchase_df = pd.concat([purchase_df,pd.DataFrame(data)], ignore_index=True)
    
    else:
        
        data = ([{'VISITOR_ID':v_id, 
         'purchase_count': 0, 
         'mean_price': 0, 
         'min_price':0, 
         'max_price':0, 
         'median_weekday': 0, 
         'median_hour': 0, 
         'median_repeat_purchase_duration':0
        }])

        purchase_df = pd.concat([purchase_df,pd.DataFrame(data)], ignore_index=True)
        
purchase_df.to_csv('purchase_df.csv')

In [407]:
activity_df = pd.DataFrame()
for i in range(0, len(customers)):
    v_id = customers.iloc[i]["VISITOR_ID"]
    
    view_history = json_normalize(json.loads(customers.iloc[i]["PAGE_VIEW_SUMMARIES"]))

    view_history["next"] = ([ datetime.strptime(i.split(".")[0], '%Y-%m-%dT%H:%M:%S')  
                                  for i in view_history["requested_at"]])
    view_history["previous"] = view_history["next"].shift(periods =1)
    view_history = view_history.iloc[1:]
    view_history["diff"] = view_history["next"] - view_history["previous"]
    view_history["diff"] = view_history["diff"].dt.total_seconds()

    view_history = view_history[view_history["diff"]<(60*30)]
    view_history = view_history[view_history["page_id"] != 'other']
    
    action_summary = view_history.groupby("page_id")["diff"].agg("sum").reset_index()
    action_summary.columns = ["activity", "seconds_on_activity"]
    action_summary = (pd.pivot_table(action_summary, values="seconds_on_activity", 
                                     columns=["activity"]).reset_index(drop=True))

    action_summary["VISITOR_ID"] = v_id
    
    activity_df = pd.concat([activity_df,action_summary], ignore_index=True)

activity_df = activity_df.fillna(0)
activity_df.to_csv('activity_df.csv')

In [470]:
customers = customers.merge(purchase_df, on='VISITOR_ID', how='left')
customers = customers.merge(activity_df, on='VISITOR_ID', how='left')

In [ ]:
customers = customers.drop(columns=['REQUESTED_AT', 'PAGE_VIEW_TYPE', 'PAGE_ID', 'PAGE_VIEW_SUMMARIES', 'CONVERSION_SUMMARIES', 'CONVERSION_SUMMARIES_len'])

In [495]:
customers.columns

Index(['VISITOR_ID', 'BROWSER_FAMILY', 'BROWSER', 'OS_FAMILY', 'OS',
       'DEVICE_FAMILY', 'FIRST_SEEN', 'future_conversion_flag',
       'returing_customer_flag', 'future_purchase_type_flag',
       'latest_visit_date', 'first_visit_date', 'cart', 'collections.hair',
       'collections.other', 'collections.paged', 'collections.products',
       'collections.tools', 'addtocart', 'collections.newarrival', 'home',
       'account', 'checkout', 'confirmation', 'orders.other', 'pages',
       'products.other', 'search', 'collections.oki',
       'collections.bestsellers', 'collections.stationary', 'collections.bath',
       'collections.socks', 'collections.makeup', 'collections', 'policies',
       'blogs', 'purchase_count', 'mean_price', 'min_price', 'max_price',
       'median_weekday', 'median_hour', 'median_repeat_purchase_duration'],
      dtype='object')

### Feature Selection

I was able to generate 42 features. 

I didn't touch the device-related information since it was already organized and wouldn't require much effort to convert it from a categorical feature to a numeric one. Instead, I decided to dig into user browsing behaviour and previous purchase history since they would better predict future purchases. 

To get a better score, I will reduce the feature set by combining the existing numeric features. I can use PCA or backwards-elimination techniques, but with PCA, it will create new features, which I want to avoid due to reduced interpretability. As for training a model that would then indicate feature importance, this would reduce the feature set but not meet the assignment goal of creating 'good quality features.' So, for this reason, I will rely on simple correlation to identify similar features and then make a new feature out of them using simple arithmetic, thereby reducing features while maintaining interpretability.  

In [525]:
# Flag any highly coorelated features
import matplotlib.pyplot as plt

numeric_features = customers[[ 
       'returing_customer_flag',
       'cart', 'collections.hair',
       'collections.other', 'collections.paged', 'collections.products',
       'collections.tools', 'addtocart', 'collections.newarrival', 'home',
       'account', 'checkout', 'confirmation', 'orders.other', 'pages',
       'products.other', 'search', 'collections.oki',
       'collections.bestsellers', 'collections.stationary', 'collections.bath',
       'collections.socks', 'collections.makeup', 'collections', 'policies',
       'blogs', 'purchase_count', 'mean_price', 'min_price', 'max_price',
       'median_weekday', 'median_hour', 'median_repeat_purchase_duration', 'future_conversion_flag']]


In [532]:
cor_data = pd.DataFrame()

for i in list(numeric_features.columns):
    for j in list(numeric_features.columns):

        cor_val = numeric_features[i].corr(numeric_features[j])
        
        data = pd.DataFrame([{'i': i, 'j': j, 'cor_val':cor_val}])

        cor_data = pd.concat([cor_data,data], ignore_index=True)
        

In [566]:
cor_data[(cor_data["cor_val"] >0.8) & (cor_data["cor_val"] <= 0.99)]
#cor_data[(cor_data["cor_val"] >0.3) & (cor_data["cor_val"] <= 0.32)]
#cor_data[cor_data['i']=='future_conversion_flag'].sort_values('cor_val')#.reset_index()

,i,j,cor_val
30,returing_customer_flag,median_weekday,0.813256
31,returing_customer_flag,median_hour,0.832022
946,mean_price,min_price,0.979926
947,mean_price,max_price,0.966641
979,min_price,mean_price,0.979926
981,min_price,max_price,0.904282
1013,max_price,mean_price,0.966641
1014,max_price,min_price,0.904282
1020,median_weekday,returing_customer_flag,0.813256
1054,median_hour,returing_customer_flag,0.832022


### Final Features

##### To separate entries
- 'VISITOR_ID'
- 'latest_visit_date'

##### Target metric
- 'future_conversion_flag'


##### Device Related - I mostly left these untouched due to the time limit. 
If I had more time, I would have dug more into each of these features and determined if any of these features have a strong association with purchases made. ie, 80% of purchases made are on a smartphone, so I would convert the phone inside of device_family into a separate feature flag. 
- 'BROWSER_FAMILY'
- 'BROWSER'
- 'OS_FAMILY'
- 'OS'
- 'DEVICE_FAMILY'
    
##### Time related
- 'first_visit_date'

##### Time spent on page activity per session
Either add up all associated features into a single column or keep the highly correlated column. 
- 'pages' - Moderately coorelated with 'products.other', 'cart'
- 'collections.other'- Moderately correlated with 'addtocart', 'collections.paged', 'collections.products', 'collections.newarrival', 'home'
- 'confirmation' - Highly correlated with 'purchase_count', 'checkout'
- 'orders.other' - moderately correlated with  'account'

##### Weakly correlated,
Add up time from the below features in the new feature "browsing_time"
- 'collections.hair'
- 'collections.tools'
- 'home'
- 'checkout'
- 'search'
- 'collections.oki'
- 'collections.bestsellers'
- 'collections.stationary'
- 'collections.bath'
- 'collections.socks'
- 'collections.makeup'
- 'collections'
- 'policies'
- 'blogs'

##### Historical Purchases
- 'mean_price' -  'max_price' will be dropped since they are highly correlated with mean price.
- 'min_price' - Highly correlated with 'returing_customer_flag'
- 'median_weekday' - Highly correlated with target metric
- 'median_hour' - Highly correlated with target metric
- 'median_repeat_purchase_duration' - Moderately correlated with target metric